# <center> Euristiche Local Search e Tabu Search per Set Covering Problem </center> #

| Cognome | Nome | Matricola |
| :-: | :-: | :-: |
| Fazzari | Daniele | M63001384 |
| Conte | Alfonso | M63001378 |

## <center>Set Covering Problem</center>

Il problema del set covering è un particolare problema di **selezione di sottoinsiemi (Subsets $S_j$**) nel quale si vuole garantire che ogni elemento appartenente all'insieme degli elementi da "coprire" (con i subsets) sia "coperto" da almeno un subset. Dunque, a differenza dei problemi di Set Partitioning e Set Packing, il generico elemento i-esimo può essere coperto da (o equivalentemente può "appartenere" a) più subset **$S_j \in Sol$** (Sol è una generica soluzione del problema).

Sia **A** la **matrice di copertura** (di dimensioni **mxn**, dove "m" è il numero di elementi da coprire ed "n" il numero di subsets) il cui generico elemento **$a_{ij}$** è definito come segue:

- **1** se i $\in S_j$ </center>
- **0** altrimenti
    
Sia **x** il **vettore delle variabili** binarie (una per ogni subset $S_j$) la cui componente j-esima **$x_j$** è definita come segue:
- **1** se $S_j \in$ Sol
- 0 altrimenti

Sia **c** il **vettore dei costi** il cui elemento j-esimo **$c_j$** rappresenta il costo associato alla scelta del subset $S_j$. Esso risulta necessario alla definizione della **funzione obiettivo**, ovvero la minimizzazione del costo complessivo della soluzione: 

\begin{equation}
\sum_{j \in n} c_j x_j 
\end{equation}

Dunque è possibile formulare un problema di Set Covering nella seguente maniera compatta:

\begin{equation}
min \;\; c^{T} x \\
A x \geq 1 \\
x \in \{0,1\}^n \\
\end{equation}

## <center> Risoluzione all'ottimo (Gurobi) </center>

Importazione degli elementi di libreria di **Gurobi Optimizer** utili alla definizione del modello e alla risoluzione del problema (GRB), importazione della libreria numpy per l'algebra lineare. I dati del problema sono contenuti in un file di testo che deve essere essere opportunamente aperto (funzione open) e interpretato successivamente.

In [25]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import time 
import matplotlib.pyplot as plt
import math 

### <center> Definizione del modello </center>

Lettura delle dimensioni m (num_righe) e n (num_colonne) della matrice di copertura A e degli elementi $a_ij$ (letti ed inseriti in una lista di interi) usati per popolarla. Definizione del vettore dei costi associati ai subsets.

In [26]:
file = open("0341.3.spp") #apertura del file contente i dati del modello
linea= file.readline() #Lettura riga con le dimensioni di A
riga_0=linea.split() 
num_righe=int(riga_0[0]) 
num_colonne=int(riga_0[1]) 
num_non_zero_elem=(riga_0[2])
print(num_righe,num_colonne)

costi_Sj = []
A = np.zeros((num_righe, num_colonne))
for i in range(num_colonne):
    linea = file.readline().split()
    costi_Sj.append(float(linea[0]))
    for k in range(1,len(linea)):
        A[int(linea[k])][i] = 1
        
A=A.tolist()

658 45800


Inizializzazione del modello, creazione delle **variabili decisionali** $x_j$ del problema, definizione della **funzione obiettivo** e aggiunta dei constraints del problema.

In [24]:
#Inizializzazione modello 
mod = gp.Model('SetCoveringOttimo')

#vettore variabili decisionali
num_Xvars=range(num_colonne)
Xvars= mod.addVars(num_Xvars, name='X', vtype=GRB.BINARY)

#Funzione obiettivo
obj = gp.quicksum(costi_Sj[i]*Xvars[i] for i in num_Xvars) 
mod.setObjective(obj, GRB.MINIMIZE)

#Vincoli del problema

Constr= mod.addConstrs((Xvars.prod(A[i])>=1 for i in range(num_righe)))

KeyboardInterrupt: 

### <center> Risoluzione ottima </center>

Tentativo di Risoluzione all'ottimo del problema (appena modellato) utilizzando il modulo **optimize**

In [4]:
mod.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads


GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

In [ ]:
if mod.status == GRB.OPTIMAL:
    foundOptimalSol = True
    Valore_Ottimo=mod.objVal
    print('Valore soluzione: %g' % Valore_Ottimo)
    X = mod.getAttr('x', Xvars)
    print(type(X))
    PrintX = np.zeros(num_colonne)
    #for i in num_Xvars:
        #if X[i] > 0 :
        #print('X(%s) = %g' % (i, X[i]) )
    print("Soluzione: ", X)

## <center> Risoluzione con Euristica Local Search </center>

### <center> Funzioni di utilità </center>

Funzione per determinazione di una soluzione ammissibile. Restituisce True se la soluzione (la quale ha associata il vettore di copertura righe in ingresso) è ammissibile per il problema di set covering considerato, False altrimenti. 
( NOTA: il nome deriva dal fatto che è utilizzata dall'algoritmo greedy )

In [27]:
def is_a_solution_greedy(righe_coperte):
    somma=np.sum(righe_coperte)
    if somma == len(righe_coperte): 
        return True
    else:
        return False

Funzione per determinazione di una soluzione ammissibile a partire dal vettore di copertura delle righe. Restituisce False se è trovata una riga  non coperta (aka un elemento non coperto)

In [28]:
def is_a_solution(righe_coperte, num_righe): 
    for i in range(num_righe):
        if righe_coperte[i]==0:
            return False
    return True

Funzione per valutare la copertura delle righe. Restituisce una lista il 
cui elemento i-esimo è il numero di subset Sj che coprono l'elemento i (ovvero la riga i)

In [29]:
def copri_righe(Sol,A, num_righe, num_colonne):
    righe_coperte = np.zeros(num_righe)
    for i in range(num_colonne):
        if(Sol[i]==1):
            for j in range(num_righe):
                if A[j][i]==1:
                    righe_coperte[j]+=1
    return righe_coperte


def copri_righe2(Sol,A, num_righe, num_colonne,righe_coperte,colonne_coprenti):
    for i in range(num_colonne):
        if(Sol[i]==1):
            for j in range(num_righe):
                if A[j][i]==1:
                    righe_coperte[i].append(j)
                    colonne_coprenti[j].append(i)

Definizione funzioni per aggiustamento del vettore di copertura delle righe in caso di eliminazione, aggiunta e swap di righe (aka elementi)

In [30]:
def delete_righe_coperte(righe_coperte,k,A,num_righe):
    righe_coperte_delete = np.zeros(num_righe)
    for i in range(num_righe):
        righe_coperte_delete[i] = righe_coperte[i]
        if A[i][k]==1:
            righe_coperte_delete[i]-=1
    return righe_coperte_delete
            
def swap_righe_coperte(righe_coperte,h,k,A,num_righe):
    righe_coperte_swap = np.zeros(num_righe)
    for i in range(num_righe):
        righe_coperte_swap[i]=righe_coperte[i]
        if A[i][h]==1:
            righe_coperte_swap[i]-=1
        if A[i][k]==1:
            righe_coperte_swap[i]+=1
    return righe_coperte_swap

def add_righe_coperte(righe_coperte,k,A,num_righe):
    righe_coperte_delete=np.zeros(num_righe)
    for i in range(num_righe):
        righe_coperte_delete[i]=righe_coperte[i]
        if A[i][k]==1:
            righe_coperte_delete[i]+=1
    return righe_coperte_delete

Definizione funzione per aggiunta alla soluzione del subset Sj con il minore rapporto costo/copertura. Il parametro min_index è di uscita, rappresenta l'indice del subset con il rapporto costo/copertura minore

In [31]:
def aggiungi_set_migliore(Sol,costi_Sj,coperture_Sj,min_index, num_colonne):
    costi=np.zeros(num_colonne)
    costi_copertura=np.zeros(num_colonne)
    for i in range(num_colonne):
        if Sol[i]==0:
            costi_copertura[i]=costi_Sj[i]/coperture_Sj[i]
    min_index=costi_copertura.tolist().index(np.min(costi_copertura[np.nonzero(costi_copertura)]))
    Sol[min_index]=1

Definizione funzione per il calcolo del valore di una soluzione, fornito il vettore soluzione e il vettore dei costi dei subset (costi_Sj)

In [32]:
def val_sol(Sol,costi_Sj):
    val=np.dot(Sol,costi_Sj)
    return val
def val_sol_elimina(val_sol_iniziale,k,costi_Sj):
    new_val=val_sol_iniziale-costi_Sj[k]
    return new_val
def val_sol_swap(val_sol_iniziale,h,k,costi_Sj):
    new_val=val_sol_iniziale-costi_Sj[h]+costi_Sj[k]
    return new_val
def val_sol_add(val_sol_iniziale,k,costi_Sj):
    new_val=val_sol_iniziale+costi_Sj[k]
    return new_val

### <center> Algoritmo Greedy </center>

Definizione di un algoritmo greedy (algoritmo costruttivo) per la ricerca della soluzione iniziale di un problema, dati in ingresso il vettore soluzione di partenza (parametro di uscita), il vettore dei costi dei subset Sj, il vettore delle righe coperte da ogni Sj nella matrice di copertura. 

In [33]:
def Algoritmo_Greedy(Sol_attuale,A,costi_Sj,coperture_Sj, num_righe, num_colonne):
    costi_copertura={}
    for i in range(num_colonne):
        costi_copertura[i]=costi_Sj[i]/coperture_Sj[i]   
    costi_copertura_sorted=sorted(costi_copertura.items(), key=lambda x:x[1])
    righe_coperte=np.zeros(num_righe)
    
    while not(is_a_solution_greedy(righe_coperte)):
        #aggiungi_set_migliore(Sol_attuale,costi_Sj,coperture_Sj,index, num_colonne)
        index=costi_copertura_sorted.pop(0)[0]
        Sol_attuale[index]=1
        for k in range(num_righe):
            if(A[k][index]==1):
                righe_coperte[k]=1
                
#DEBUG
Soluzione_Greedy = np.zeros(num_colonne) #Vettore atto a contenere la soluzione Greedy
coperture_Sj = np.sum(A, axis=0) #L'elemento j di coperture è la somma di A[i][j] per ogni i
temp = coperture_Sj.tolist().copy()
st = time.time()
Algoritmo_Greedy(Soluzione_Greedy,A,costi_Sj,temp, num_righe, num_colonne)
et = time.time()
elapsed_time = et - st
print("Soluzione_Greedy:",Soluzione_Greedy)
print("Valore:", val_sol(Soluzione_Greedy,costi_Sj))
print('Execution time Algoritmo Greedy:', elapsed_time,"s")

Soluzione_Greedy: [1. 1. 1. ... 0. 1. 0.]
Valore: 7185043.000000006
Execution time Algoritmo Greedy: 7.821327209472656 s


### <center> Euristica di Ricerca Locale </center> 

Definizione delle mosse per la realizzazione di un algoritmo di Ricerca Locale. Mosse:
- **Eliminazione** di un subset dalla soluzione attuale
- **Aggiunta** di un subset alla soluzione attuale
- **Scambio** di un subset non appartente alla soluzione attuale con uno appartente ad essa

In [34]:
def elimina_set(Sol,k):
    Sol[k]=0
    
def aggiungi_set(Sol,k):
    Sol[k]=1
    
def scambia_set(Sol,h,k):
    elimina_set(Sol,h)
    aggiungi_set(Sol,k)

Procedura di definizione dell'**intorno** nello spazio delle soluzioni, determinato utilizzando iterativamente le mosse precedentemente definite. Tale procedura è utilizzata dalla euristica di ricerca locale per ottenere l'insieme delle soluzioni ammissibili all'interno del quale cercare una soluzione migliore della attuale.

In [35]:
def intorno(Sol,A,num_righe,num_colonne,costi_Sj, m,):
    
    righe_coperte_base=np.zeros(num_righe)
    righe_coperte_base=copri_righe(Sol,A, num_righe, num_colonne)
    soluzioni_vicine=[]
    val_sol_iniziale=val_sol(Sol,costi_Sj)
    #copri_righe2(Sol,A, num_righe, num_colonne,righe_coperte,colonne_coprenti)
    righe_coperte_delete=[]
    righe_coperte_add=[]
    righe_coperte_delete=righe_coperte_base.copy()
    righe_coperte_swap=righe_coperte_base.copy()
    num_soluzioni_delete=0
    num_soluzioni_swap=0
    l=0
    j=0
    while(num_soluzioni_delete<10 and l<m):
        new_sol=Sol.copy()
        new_val=val_sol_iniziale
        for i in range(int(num_colonne/100)):
            k=np.random.randint(num_colonne)
            if(new_sol[k]!=0):
                elimina_set(new_sol,k)
                righe_coperte_delete=delete_righe_coperte(righe_coperte_delete,k,A,num_righe)
                new_val=val_sol_elimina(new_val,k,costi_Sj)
                if(is_a_solution(righe_coperte_delete, num_righe)):
                    soluzioni_vicine.append((new_sol.copy(),new_val))
                    num_soluzioni_delete+=1;
        l+=1;

            
    while(num_soluzioni_swap<50 and j<m):
        new_sol2=Sol.copy()
        new_val2=val_sol_iniziale
        for t in range(int(num_colonne/100)):
            k=np.random.randint(num_colonne)
            h=np.random.randint(num_colonne)
            if(new_sol2[h]!=0 and new_sol2[k]!=1):
                #new_sol2=Sol.copy()
                scambia_set(new_sol2,h,k)
                righe_coperte_swap=swap_righe_coperte(righe_coperte_swap,h,k,A,num_righe)
                new_val2=val_sol_swap(new_val2,h,k,costi_Sj)
                if(is_a_solution(righe_coperte_swap, num_righe)):          
                    soluzioni_vicine.append((new_sol2.copy(),new_val2))
                    num_soluzioni_swap+=1
        j+=1
            
    return soluzioni_vicine

Definizione della **ricerca locale** con tecnica **BEST IMPROVEMENT**. Una volta valutato l'intorno della soluzione corrente, viene aggiornata la soluzione attuale con la soluzione ammissibile migliore nell'intorno. Tale soluzione ammissibile selezionata deve avere un valore, in tal caso un valore di funzione obiettivo (estimatore), minore (trattandosi di un problema a minimizzazione del costo) della soluzione a partire dalla quale è stato costruito l'intorno, in quanto non si accettano delle iterazioni peggiorative dell'algoritmo stesso. E' da notare che si tratta dunque di una **euristica MIGLIORATIVA**, in quanto, partendo da una soluzione ammissibile, cerca dei miglioramenti di essa al fine di trovare una soluzione ammissibile definitiva migliore.

In [36]:
def ricerca_locale(Sol,A,num_righe,num_colonne,costi_Sj,val_soluzioni,continua=True):
    
    vecchia_soluzione=Sol
    valore_sol = val_sol(Sol,costi_Sj)
    iter=0
    stessa_sol=0
    soluzioni=[]
    
    #Dimensione dell'intorno dipendente dalla dimensione del problema
    dim_intorno = num_colonne/20
    
    while True:
        
        soluzioni_vicine = intorno(vecchia_soluzione,A,num_righe,num_colonne,costi_Sj,dim_intorno)    
        val_migliore_sol = val_sol(vecchia_soluzione,costi_Sj)
        migliore_sol=vecchia_soluzione

        if continua and iter%5==0:
            print("iterazione N°:",iter, "--> Miglioramento Percentuale:", int(val_migliore_sol/valore_sol*100), "%")
            
            if ((val_migliore_sol/valore_sol*100)<=30):
                break
           # print("val_migliore_sol",val_migliore_sol,"valore_sol",valore_sol)
        for sol,val in soluzioni_vicine:
            
            if(val < val_migliore_sol):
                #print("VAL E' < DI VAL_MIGLIORE_SOL",val,"  ",val_migliore_sol)
                val_migliore_sol = val
                migliore_sol = sol
                val_soluzioni.append(val)
                
                
        
        if((migliore_sol == vecchia_soluzione).all()):
            stessa_sol = stessa_sol + 1
            if(stessa_sol == 50):
                break
        else:
            stessa_sol = 0
            vecchia_soluzione = migliore_sol
            

        iter = iter+1
        
    return vecchia_soluzione

In [37]:

val_soluzioni=[]
t1=time.time()
Soluzione_Local=ricerca_locale(Soluzione_Greedy,A,num_righe,num_colonne,costi_Sj,val_soluzioni)
t2=time.time()

    
print("Soluzione --> ",Soluzione_Local,"\nValore soluzione: ",val_sol(Soluzione_Local,costi_Sj))
print("Execution time Ricerca locale: ",t2-t1, "s")  

iterazione N°: 0 --> Miglioramento Percentuale: 100 %
iterazione N°: 5 --> Miglioramento Percentuale: 95 %
iterazione N°: 10 --> Miglioramento Percentuale: 90 %
iterazione N°: 15 --> Miglioramento Percentuale: 86 %
iterazione N°: 20 --> Miglioramento Percentuale: 81 %
iterazione N°: 25 --> Miglioramento Percentuale: 78 %
iterazione N°: 30 --> Miglioramento Percentuale: 74 %
iterazione N°: 35 --> Miglioramento Percentuale: 70 %
iterazione N°: 40 --> Miglioramento Percentuale: 67 %
iterazione N°: 45 --> Miglioramento Percentuale: 63 %
iterazione N°: 50 --> Miglioramento Percentuale: 60 %
iterazione N°: 55 --> Miglioramento Percentuale: 57 %
iterazione N°: 60 --> Miglioramento Percentuale: 54 %
iterazione N°: 65 --> Miglioramento Percentuale: 52 %
iterazione N°: 70 --> Miglioramento Percentuale: 49 %
iterazione N°: 75 --> Miglioramento Percentuale: 47 %
iterazione N°: 80 --> Miglioramento Percentuale: 44 %
iterazione N°: 85 --> Miglioramento Percentuale: 42 %
iterazione N°: 90 --> Miglior

In [ ]:
a=np.array(val_soluzioni)
l=len(a)
Best=val_sol(Soluzione_Greedy,costi_Sj) 
plt.title("Local Search",fontsize=25)
plt.ylabel("Valore funzione obiettivo")
plt.xlabel("Iterazione")
plt.xlim(-num_colonne/2, l+num_colonne/2)
plt.ylim(min(a)-num_colonne/2, max(a)+num_colonne/2)
plt.plot([l-1],[Best], 'ro', label='Valore Greedy')
for i in range(l):
    plt.plot([i],a[i],'b+')
plt.legend()

Definizione di una euristica di ricerca locale molto simile alla precedente, ma che implementa la ricerca nell'intorno con 
**tecnica FIRST IMPROVEMENT**. Verosimilmente dovrebbe verificarsi un calo nel tempo complessivo di calcolo della soluzione sub-ottima, con un conseguente (ma non certo) peggioramento del valore della soluzione finale.

In [ ]:
def ricerca_locale(Sol,A,num_righe,num_colonne,costi_Sj,val_soluzioni,continua=True):
    
    vecchia_soluzione=Sol
    valore_sol = val_sol(Sol,costi_Sj)
    iter=0
    stessa_sol=0
    soluzioni=[]
    
    #Dimensione dell'intorno dipendente dalla dimensione del problema
    dim_intorno = num_colonne/20
    
    while True:
        
        soluzioni_vicine = intorno(vecchia_soluzione,A,num_righe,num_colonne,costi_Sj,dim_intorno)    
        val_migliore_sol = val_sol(vecchia_soluzione,costi_Sj)
        migliore_sol=vecchia_soluzione

        if continua and iter%5==0:
            print("iterazione N°:",iter, "--> Miglioramento Percentuale:", int(val_migliore_sol/valore_sol*100), "%")
            
            if ((val_migliore_sol/valore_sol*100)<=30):
                break
           # print("val_migliore_sol",val_migliore_sol,"valore_sol",valore_sol)
        for sol,val in soluzioni_vicine:
            
            if(val < val_migliore_sol):
                #print("VAL E' < DI VAL_MIGLIORE_SOL",val,"  ",val_migliore_sol)
                val_migliore_sol = val
                migliore_sol = sol
                val_soluzioni.append(val)        
                break
                      
        
        if((migliore_sol == vecchia_soluzione).all()):
            stessa_sol = stessa_sol + 1
            if(stessa_sol == 50):
                break
        else:
            stessa_sol = 0
            vecchia_soluzione = migliore_sol
            

        iter = iter+1
        
    return vecchia_soluzione

In [17]:
#DEBUG
val_soluzioni=[]
t1=time.time()
Soluzione_Local_firts_improvement=ricerca_locale_first_improvement(Soluzione_Greedy,A,num_righe, num_colonne, costi_Sj,val_soluzioni)
t2=time.time()
print("Soluzione --> ",Soluzione_Local_firts_improvement,"\nValore soluzione: ",val_sol(Soluzione_Local_firts_improvement,costi_Sj))
print("Execution time Ricerca locale: ",t2-t1, "s")  

NameError: name 'ricerca_locale_first_improvement' is not defined

## <center> Euristica Tabu Search </center>

Classe coda_tabu per implementare il **TDA** per gestire le **mosse tabu**. L'elemento i-esimo della coda conterrà un intero rappresentante l'indice del subset Sj (nel vettore soluzione) che è stato oggetto di una mossa. Trattandosi di una **coda circolare** un determinato elemento non farà più parte della coda dopo dim_coda inserimenti. 

In [ ]:
class coda_tabu:

    #dimensione = numero elementi contenuti
    def __init__(self, dimensione=5):
        self.coda = [None] * dimensione  #coda
        self.dimensione_max = dimensione
        self.punt_testa = 0 
        self.punt_coda = -1
        self.riemp = 0

    def inserisci_elemento_tabu(self, elem_tabu):
        self.punt_coda = (self.punt_coda + 1) % self.dimensione_max
        self.coda[self.punt_coda] = elem_tabu
        if(self.riemp!=self.dimensione_max):
            self.riemp = self.riemp + 1

    def trova_elemento(self, elem_tabu): 
        for i in range(self.riemp):
            if(self.coda[i] == elem_tabu):
                return True
        return False   

    def stampa_coda(self):
        print("Print Coda: ")
        for i in range(self.riemp):
            print(self.coda[i])
            
    def pulisci_tabu(self):
        for i in range(self.riemp):
            self.coda[i]=None
        self.riemp=0
        self.punt_testa = 0 
        self.punt_coda = -1

Codice di definizione dell'intorno per la **tabu search**. A differenza della procedura di calcolo dell'intorno per la ricerca locale standard, questa, nonostante generi indici casuali per determinare i subset da eliminare/aggiungere, effettua la mossa esclusivamente se l'indice del subset soggetto della mossa non si trova nella coda circolare tabu.

In [ ]:
def intorno_tabu(Sol,A,num_colonne, costi_Sj,coda_tabu, dim_intorno):
    
    m=dim_intorno
    righe_coperte_base=np.zeros(num_righe)
    righe_coperte_base=copri_righe(Sol,A, num_righe, num_colonne)
    soluzioni_vicine=[]
    val_sol_iniziale=val_sol(Sol,costi_Sj)
    righe_coperte_delete=[]
    righe_coperte_add=[]
    righe_coperte_delete=righe_coperte_base.copy()
    righe_coperte_swap=righe_coperte_base.copy()
    num_soluzioni_delete=0
    num_soluzioni_swap=0
    l=0
    j=0
    while(num_soluzioni_delete<10 and l<m):
        new_sol=Sol.copy()
        new_val=val_sol_iniziale
        for i in range(num_colonne/100):
            k=np.random.randint(num_colonne)
            if(new_sol[k]!=0 and not(coda_tabu.trova_elemento(k))):
                elimina_set(new_sol,k)
                righe_coperte_delete=delete_righe_coperte(righe_coperte_delete,k,A,num_righe)
                new_val=val_sol_elimina(new_val,k,costi_Sj)
                if(is_a_solution(righe_coperte_delete, num_righe)):
                    soluzioni_vicine.append((new_sol.copy(),new_val,(k,-1)))
                    num_soluzioni_delete+=1;
            l+=1;

            
    while(num_soluzioni_swap<50 and j<m):
        new_sol2=Sol.copy()
        new_val2=val_sol_iniziale
        for t in range(num_colonne/100):
            k=np.random.randint(num_colonne)
            h=np.random.randint(num_colonne)
            if(new_sol2[h]!=0 and new_sol2[k]!=1 and not(coda_tabu.trova_elemento(h)) and not(coda_tabu.trova_elemento(k))):
                scambia_set(new_sol2,h,k)
                righe_coperte_swap=swap_righe_coperte(righe_coperte_swap,h,k,A,num_righe)
                new_val2=val_sol_swap(new_val2,h,k,costi_Sj)
                if(is_a_solution(righe_coperte_swap, num_righe)):          
                    soluzioni_vicine.append((new_sol2.copy(),new_val2,(k,h)))
                    num_soluzioni_swap+=1
        j+=1
                

    return soluzioni_vicine

Implementazione della Euristica Tabu Search. All'interno di tale implementazione si ritrovano i concetti di **intensificazione** (con l'aumento della dimensione e della popolosità dell'interno da ispezionare) e **diversificazione** (con la possibilità di effettuare delle iterazioni peggiorative

In [ ]:
def tabu_search(Sol,A,num_colonne, costi_Sj,val_soluzioni,continua=True):
    
    vecchia_soluzione=Sol
    valore_sol = val_sol(Sol,costi_Sj)
    iter=0
    stessa_sol=0
    soluzioni=[]
        
    #Dimensione dell'intorno dipendente dalla dimensione del problema
    dim_intorno = num_colonne/20
    coda_tb=coda_tabu(int(num_colonne*0.06))
    
    while True:
        
        soluzioni_vicine = intorno_tabu(vecchia_soluzione,A,num_colonne,costi_Sj,coda_tb,dim_intorno)    
        val_migliore_sol = val_sol(vecchia_soluzione,costi_Sj)
        migliore_sol=vecchia_soluzione

        if continua and iter%5==0:
            print("iterazione N°:",iter, "--> Miglioramento Percentuale:", int(val_migliore_sol/valore_sol*100), "%")
            
            if ((val_migliore_sol/valore_sol*100)<=30):
                break
           # print("val_migliore_sol",val_migliore_sol,"valore_sol",valore_sol)
        for sol,val in soluzioni_vicine:
            
            if(val < val_migliore_sol):
                #print("VAL E' < DI VAL_MIGLIORE_SOL",val,"  ",val_migliore_sol)
                val_migliore_sol = val
                migliore_sol = sol
                val_soluzioni.append(val)
                
                
        
        if((migliore_sol == vecchia_soluzione).all()):
            stessa_sol = stessa_sol + 1
            if(stessa_sol == 10):
                break
        else:
            stessa_sol = 0
            vecchia_soluzione = migliore_sol
            

        iter = iter+1
        
    return vecchia_soluzione

In [ ]:
val_soluzioni2 = []
soluzione = np.ones(num_colonne)
t1=time.time()
Soluzione_tabu=tabu_search(Soluzione_Greedy,A,num_colonne,costi_Sj,val_soluzioni2)
t2=time.time()
print("Soluzione : ",Soluzione_tabu,"\nValore soluzione: ",val_sol(Soluzione_tabu,costi_Sj),"\nExecution time Tabu Search:",t2-t1," s")

In [ ]:
a=np.array(val_soluzioni2)
l=len(a)
Best=Valore_Ottimo #Valore ottenuto dall' esecuzione dell'algoritmo esatto con Gurobi
plt.title("Tabu Search",fontsize=25)
plt.ylabel("Valore funzione obiettivo")
plt.xlabel("Iterazione")
plt.xlim(-num_colonne/4, l+num_colonne/4)
plt.ylim(min(a)-num_colonne/4, max(a)+num_colonne/4)
plt.plot([l-1],[Best], 'ro', label='Valore Greedy')
for i in range(l):
    plt.plot([i],a[i],'b+')
plt.legend()